In [5]:
import sqlite3
import pandas as pd # Using pandas to clean up appearances of query results
def sql(query): # defined function because I'm lazy and didn't want keep typing methods
    connection = sqlite3.connect('sqlite_db_pythonsqlite.db')
    cur = connection.cursor().execute
    return pd.DataFrame(cur(query).fetchall())

sql('Select * from Facilities') #Function Test

,0,1,2,3,4,5
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


**Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!**

In [11]:
db=sql("""SELECT f.name,
    SUM(CASE 
        WHEN b.memid <>0 
            THEN f.membercost*b.slots
        ELSE f.guestcost*b.slots
        END) AS revenue
    FROM Bookings as b
    INNER JOIN Facilities as f
        ON b.facid = f.facid
    GROUP BY f.name
        HAVING revenue > 1000
    ORDER BY revenue DESC;
        """)
db.rename(columns={0:'Facility', 1:'Total Revenue'})

,Facility,Total Revenue
0,Massage Room 1,50351.6
1,Massage Room 2,14454.6
2,Tennis Court 2,14310.0
3,Tennis Court 1,13860.0
4,Squash Court,13468.0
5,Badminton Court,1906.5


**Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order**

In [14]:
db=sql("""
    SELECT m.surname, 
    m.firstname, 
    m.recommendedby AS recomender_id, 
    r.surname || " " ||r.firstname  AS 'Recommended By'
    FROM Members AS m
    LEFT JOIN Members AS r 
        ON m.recommendedby = r.memid
    WHERE m.recommendedby != 0
    ORDER BY r.surname;""")
db.rename(columns={0:'Last Name', 1:'First Name', 2:'Recommending ID', 3:'Recommended By'})

,Last Name,First Name,Recommending ID,Recommended By
0,GUEST,GUEST,,None
1,Smith,Darren,,None
2,Smith,Tracy,,None
3,Rownam,Tim,,None
4,Tracy,Burton,,None
5,Farrell,Jemima,,None
6,Farrell,David,,None
7,Tupperware,Hyacinth,,None
8,Smith,Darren,,None
9,Sarwin,Ramnaresh,15,Bader Florence


**Q12: Find the facilities with their usage by member, but not guests**

In [12]:
db=sql("""
SELECT b.facid, 
    COUNT(b.memid) AS 'usage', 
    f.name
FROM 
    (SELECT facid, memid FROM Bookings WHERE memid <>0) AS b
LEFT JOIN Facilities AS f
    ON b.facid = f.facid
GROUP BY b.facid;
""")
db.rename(columns={0:"Facility ID", 1:'Usage', 2:'Facility'})

,Facility ID,Usage,Facility
0,0,308,Tennis Court 1
1,1,276,Tennis Court 2
2,2,344,Badminton Court
3,3,385,Table Tennis
4,4,421,Massage Room 1
5,5,27,Massage Room 2
6,6,195,Squash Court
7,7,421,Snooker Table
8,8,783,Pool Table


**Q13: Find the facilities usage by month, but not guests**

In [10]:
db=sql("""
    SELECT b.months,
    COUNT (b.memid) AS 'usage'
    FROM(
        SELECT strftime('%m', starttime) AS "months", memid
        FROM Bookings
        WHERE memid <>0) AS b
    GROUP BY b.months;
""")
db.rename(columns ={0:'Month', 1:'Usage'})

,Month,Usage
0,07,480
1,08,1168
2,09,1512
